In [ ]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load training images
train_images = pickle.load(open("/kaggle/input/img-lab/full_CNN_train.p", "rb"))

# Load labels
labels = pickle.load(open("/kaggle/input/img-lab/full_CNN_labels.p", "rb"))

# Keep images, labels as numpy arrays
train_images = np.array(train_images)
labels = np.array(labels)

# Normalizing the labels
labels = labels / 255

# Shuffling the images, then train into train/validation
train_images, labels = shuffle(train_images, labels)

x_train, x_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

# Batch size, epochs, pool size-can be fiddled for optimization
batch_size = 150
epochs = 20
pool_size = (2, 2)
input_shape = x_train.shape[1:]

In [ ]:
model = Sequential()
# normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=input_shape))
# conv layer 1
model.add(
    Conv2D(60, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv1")
)
# conv layer 2
model.add(
    Conv2D(50, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv2")
)
# pooling 1
model.add(MaxPooling2D(pool_size=pool_size))
# Conv layer 3
model.add(
    Conv2D(40, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv3")
)
model.add(Dropout(0.2))
# Conv layer 4
model.add(
    Conv2D(30, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv4")
)
model.add(Dropout(0.2))
# Conv layer 5
model.add(
    Conv2D(20, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv5")
)
model.add(Dropout(0.2))
# pooling 2
model.add(MaxPooling2D(pool_size=pool_size))
# Conv layer 6
model.add(
    Conv2D(10, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv6")
)
model.add(Dropout(0.2))
# Conv layer 7
model.add(
    Conv2D(5, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Conv7")
)
model.add(Dropout(0.2))
# pooling 3
model.add(MaxPooling2D(pool_size=pool_size))

# upsample 1
model.add(UpSampling2D(size=pool_size))
# Deconv 1
model.add(
    Conv2DTranspose(
        10, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Deconv1"
    )
)
model.add(Dropout(0.2))
# Deconv2
model.add(
    Conv2DTranspose(
        20, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Deconv2"
    )
)
model.add(Dropout(0.2))
# upsample 2
model.add(UpSampling2D(size=pool_size))
# Deconv 3
model.add(
    Conv2DTranspose(
        30, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Deconv3"
    )
)
model.add(Dropout(0.2))
# Deconv4
model.add(
    Conv2DTranspose(
        40, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Deconv4"
    )
)
model.add(Dropout(0.2))
# Deconv 5
model.add(
    Conv2DTranspose(
        50, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Deconv5"
    )
)
model.add(Dropout(0.2))
# upsample 3
model.add(UpSampling2D(size=pool_size))
# Deconv 6
model.add(
    Conv2DTranspose(
        60, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Deconv6"
    )
)

# Final layer--only including one channel so 1 filter
model.add(
    Conv2DTranspose(
        1, (3, 3), padding="valid", strides=(1, 1), activation="relu", name="Final"
    )
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
datagen = ImageDataGenerator()
datagen.fit(x_train)

# Compiling and training the model
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    verbose=1,
    validation_data=(x_val, y_val),
)

# Save model architecture and weights
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.weights.h5")

# Summary of model
model.summary()

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


77/77 ━━━━━━━━━━━━━━━━━━━━ 92s 751ms/step - loss: 0.0879 - val_loss: 0.0316
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 27s 341ms/step - loss: 0.0213 - val_loss: 0.0228
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 27s 349ms/step - loss: 0.0148 - val_loss: 0.0160
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 357ms/step - loss: 0.0124 - val_loss: 0.0119
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 357ms/step - loss: 0.0115 - val_loss: 0.0132
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 353ms/step - loss: 0.0108 - val_loss: 0.0110
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 357ms/step - loss: 0.0104 - val_loss: 0.0122
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 354ms/step - loss: 0.0100 - val_loss: 0.0093
Epoch 9/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 355ms/step - loss: 0.0098 - val_loss: 0.0099
Epoch 10/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 356ms/step - loss: 0.0093 - val_loss: 0.0087
Epoch 11/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 355ms/step - loss: 0.0091 - val_loss: 0.0088
Epoch 12/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 28s 355ms/ste

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization                  │ (None, 80, 160, 3)          │              12 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv1 (Conv2D)                       │ (None, 78, 158, 60)         │           1,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2 (Conv2D)                       │ (None, 76, 156, 50)         │          27,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 38, 78, 50)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv3 (Conv2D)                       │ (None, 36, 76, 40)          │          18,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 36, 76, 40)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv4 (Conv2D)                       │ (None, 34, 74, 30)          │          10,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 34, 74, 30)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv5 (Conv2D)                       │ (None, 32, 72, 20)          │           5,420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32, 72, 20)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 36, 20)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv6 (Conv2D)                       │ (None, 14, 34, 10)          │           1,810 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 14, 34, 10)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv7 (Conv2D)                       │ (None, 12, 32, 5)           │             455 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 12, 32, 5)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 16, 5)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 12, 32, 5)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Deconv1 (Conv2DTranspose)            │ (None, 14, 34, 10)          │             460 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 14, 34, 10)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Deconv2 (Conv2DTranspose)            │ (None, 16, 36, 20)          │           1,820 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 16, 36, 20)          │              

 Total params: 388,484 (1.48 MB)

 Trainable params: 129,492 (505.83 KB)

 Non-trainable params: 6 (24.00 B)

 Optimizer params: 258,986 (1011.67 KB)